# Futures

In [1]:
import os
import time
from concurrent import futures

In [4]:
WORK_LIST = [10000, 100000, 1000000, 10000000, 100000000]

In [5]:
# 동시성 합계

def sum_generator(n):
    return sum(n for n in range(1, n + 1))

## `map`

- multi Thread

In [21]:
worker = len(WORK_LIST)

start_time= time.time()

with futures.ThreadPoolExecutor() as excutor:
    result = excutor.map(sum_generator, WORK_LIST)

end_time = time.time()

print(f'time = {end_time- start_time:.3f}')

time = 4.369


- multi Process

In [22]:
worker = len(WORK_LIST)

start_time= time.time()

with futures.ProcessPoolExecutor() as excutor:
    result = excutor.map(sum_generator, WORK_LIST)

end_time = time.time()

print(f'time = {end_time- start_time:.3f}')

time = 3.742


## `wait`, `as_completed`

In [2]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed

In [6]:
start_time= time.time()

futures_list = []

with ProcessPoolExecutor() as excutor:
    for work in WORK_LIST:
        # future 반환
        future = excutor.submit(sum_generator, work)  # 여기서 바로 연산실행은 아님
        # Scheduling
        futures_list.append(future)
    # timeout을 걸어둔다
    result = wait(futures_list, timeout=3)

end_time = time.time()

print(f'time = {end_time- start_time:.3f}')

time = 3.510


In [8]:
result.done

{<Future at 0x7fe2443a8d00 state=finished returned int>,
 <Future at 0x7fe2444689d0 state=finished returned int>,
 <Future at 0x7fe244649790 state=finished returned int>,
 <Future at 0x7fe244656b20 state=finished returned int>}

In [11]:
[done_future.result() for done_future in result.done]

[50000005000000, 50005000, 5000050000, 500000500000]

In [9]:
result.not_done  # timeout=3 보다 오래걸려서 실패

{<Future at 0x7fe2441603d0 state=finished returned int>}

In [15]:
start_time= time.time()

futures_list = []

result_list = []
done_list = []
cancelled_list = []

with ProcessPoolExecutor() as excutor:
    for work in WORK_LIST:
        # future 반환
        future = excutor.submit(sum_generator, work)  # 여기서 바로 연산실행은 아님
        # Scheduling
        futures_list.append(future)
    # 먼저 처리되는 결과가 먼저 나온다
    for f in as_completed(futures_list):
        result_list.append(f.result())
        done_list.append(f.done())
        cancelled_list.append(future.cancelled())
        

end_time = time.time()

print(f'time = {end_time- start_time:.3f}')

time = 3.757


In [16]:
result_list

[50005000, 5000050000, 500000500000, 50000005000000, 5000000050000000]

In [17]:
done_list

[True, True, True, True, True]

In [18]:
cancelled_list

[False, False, False, False, False]